# Migrating to 0.2.0

The 0.2.0 release contains a handful of breaking changes compared to the previous 0.1.x releases.  This guide will list out these changes as well as how to migrate to the new features that encompass them in 0.2.x.

## Pydantic Support

In 0.1.x, Guardrails supported pydantic models with a `register_pydantic` decorator.  This decorator has been removed in 0.2.x and replaced with the `Guard.from_pydantic` method.  This method takes in a pydantic model and returns a `Guard` instance that can be used to validate the model.

To migrate from the `register_pydantic` decorator to the `Guard.from_pydantic` method, instantiate the model outside of the decorator and pass it into the `Guard.from_pydantic` method.  If you wish to continue using validators defined on the model with Guardrails, you must define them separately and register them with the `register_validator` decorator.

For example, the following 0.1.x-style rail spec defines a model with the `register_pydantic` decorator:

```xml
<rail version="0.1">

<script language="python">
from guardrails.utils.pydantic_utils import register_pydantic
from pydantic import BaseModel, validator

@register_pydantic
class Person(BaseModel):
    '''
    Information about a person.

    Args:
        name (str): The name of the person.
        age (int): The age of the person.
        zip_code (str): The zip code of the person.
    '''
    name: str
    age: int
    zip_code: str

    @validator("zip_code")
    def zip_code_must_be_numeric(cls, v):
        if not v.isnumeric():
            raise ValueError("Zip code must be numeric.")
        return v

    @validator("age")
    def age_must_be_between_0_and_150(cls, v):
        if not 0 >= v >= 150:
            raise ValueError("Age must be between 0 and 150.")
        return v

    @validator("zip_code")
    def zip_code_in_california(cls, v):
        if not v.startswith("9"):
            raise ValueError("Zip code must be in California, and start with 9.")
        if v == "90210":
            raise ValueError("Zip code must not be Beverly Hills.")
        return v

</script>

<output>
    <list name="people" description="A list of 3 people.">
        <pydantic description="Information about a person." model="Person" on-fail-pydantic="reask" />
    </list>
</output>
<prompt>
Generate data for possible users in accordance with the specification below.
@xml_prefix_prompt
{output_schema}
@complete_json_suffix_v2
</prompt>
</rail>
```

In 0.2.x, this can be migrated to the following:

In [1]:
from guardrails import Guard
from typing import Any, Dict, List

from pydantic import BaseModel, Field

from guardrails.validators import (
    FailResult,
    PassResult,
    ValidationResult,
    Validator,
    register_validator,
)


@register_validator(name="zip_code_must_be_numeric", data_type="string")
class ZipCodeMustBeNumeric(Validator):
    def validate(self, value: Any, metadata: Dict[str, Any]) -> ValidationResult:
        if not value.isnumeric():
            return FailResult(error_message="Zip code must be numeric.")
        return PassResult()


@register_validator(name="age_must_be_between_0_and_150", data_type="integer")
class AgeMustBeBetween0And150(Validator):
    def validate(self, value: Any, metadata: Dict[str, Any]) -> ValidationResult:
        if not 0 <= value <= 150:
            return FailResult(error_message="Age must be between 0 and 150.")
        return PassResult()


@register_validator(name="zip_code_in_california", data_type="string")
class ZipCodeInCalifornia(Validator):
    def validate(self, value: Any, metadata: Dict[str, Any]) -> ValidationResult:
        if not value.startswith("9"):
            return FailResult(
                error_message="Zip code must be in California, and start with 9."
            )
        if value == "90210":
            return FailResult(error_message="Zip code must not be Beverly Hills.")
        return PassResult()


class Person(BaseModel):
    """Information about a person.
    Args:
        name (str): The name of the person.
        age (int): The age of the person.
        zip_code (str): The zip code of the person.
    """

    name: str
    age: int = Field(..., validators=[AgeMustBeBetween0And150(on_fail="reask")])
    zip_code: str = Field(
        ...,
        validators=[
            ZipCodeMustBeNumeric(on_fail="reask"),
            ZipCodeInCalifornia(on_fail="reask"),
        ],
    )


class ListOfPeople(BaseModel):
    """A list of people.
    Args:
        people (List[Person]): A list of people.
    """

    people: List[Person]


prompt = """
Generate data for possible users in accordance with the specification below.
@xml_prefix_prompt
{output_schema}
@complete_json_suffix_v2
"""

guard = Guard.from_pydantic(ListOfPeople, prompt=prompt)

## Polymorphism With Discriminators In Output Schemas

## Changes To Validators (Note for Raf: This section title should be more specific)

## Removal Of Script Support

## String Formatting In Prompts
Previously Guardrails used `f-string`'s and `str.format` to include prompt variables, prompt primitives, and other partials in the prompt sent to the LLM.  This necessitated any braces, aka curly brackets, to be escaped with additional braces to prevent its contents from being considered as variable that should be substituted during string formatting.

We now use `Template.safe_substitute` to avoid this issue completely.  This does, however, require a change to how variables are expressed in string arguments such as prompts and instructions.  Now instead of expressing a variable for the prompt as `{{my_var}}`, it should be expressed as `${my_var}`.

In addition to this change in variable syntax, we have also started namespacing our prompt primitives.  Whereas before you would use one of our built in primitives like so: `@complete_json_suffix_v2`, they should now be specified as `${gr.complete_json_suffix_v2}`.

To highlight these changes in practice, below is the prompt section of a RAIL spec where each tab shows the respective before and after format.

=== "0.1.x"

```xml
<prompt>
Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 'None'.

{document}

@xml_prefix_prompt

{{output_schema}}


@json_suffix_prompt
</prompt>
```

=== "0.2.x
```xml
<prompt>
Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 'None'.

${document}

${gr.xml_prefix_prompt}

${output_schema}

${gr.json_suffix_prompt}
</prompt>
```